# 📌 Persistence

- **Definition:**
Ability to save and restore the state of a workflow over time.

Core behavior of LangGraph:
When a workflow ends, the value of the final state is deleted.

With persistence:

We can save the state.

Persistence also saves all the values from intermediate states (if any value is changed).


- **Fault tolerance:**

If any node fails during the workflow, the intermediate state value is saved.

The workflow can be restarted from where it failed.

This feature is called fault tolerance.


- **Applications:**
Helps in building chatbots.

- **Implementation:**
Persistence in LangGraph is implemented using checkpoints.



---

### 📌 Checkpoints in Persistence

- **Checkpoint Diagram:**

Workflow progresses through nodes and supersteps.

At various points (e.g., start, nodes, end), checkpoints are taken.


- **Supersteps:**

Each superstep of the graph is treated as a checkpoint.

Checkpoints save the state values for database persistence.




---

- **📌 Threads in Persistence**

Each time a workflow is executed, different thread IDs are assigned.

Using these thread IDs, state values can be retrieved later.

```
START
  |
  v
+-------+
| NODE1 |  <=== checkpoint
+-------+
  |
  v
Superstep ①
  |
  v
+-------+          +--------+
| NODE2 | <------> | NODE1  |
+-------+          +--------+
  |                  ^
  v                  |
+--------+           |
| NODE3  |-----------
+--------+
  |
  v
Superstep ②
  |
  v
+--------+
| NODE4  | <=== checkpoint
+--------+
  |
  v
Superstep ③
  |
  v
+--------+
|  I/O   |
+--------+ <=== checkpoint

Superstep ④

  ↑
  |
+--------+
| NODE1  |  <=== checkpoint
+--------+

Superstep ⑤
```

In [3]:

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # this is kind of checkpointer that helps in implementation on checkpointers, this saves all the states in RAM this is mostly used in demo projects in production we use different checkpointers

In [4]:
import os
load_dotenv()
key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=key)
llm.invoke("hello").content

'Hello. How can I assist you today?'

In [9]:
#state

class JokeState(TypedDict):
    topic: str
    joke: str
    explanation : str

In [10]:
def generate_joke(state: JokeState):
    prompt = f"write an joke on topic - {state['topic']} in 20 words"
    joke = llm.invoke(prompt).content
    return {"joke": joke}

In [11]:
def generate_explanation(state: JokeState):
    prompt = f"write explanation for the joke - {state['joke']} in 30 words"
    explanation = llm.invoke(prompt).content
    return {"explanation": explanation}

In [12]:
graph = StateGraph(JokeState)

graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", 'generate_explanation')
graph.add_edge('generate_explanation', END)

# IN ORDER TO MAKE SURE OUR WORKFLOW IMPLEMENT IMPLEMENTS PERSISTENT
#WE MAKE A CHECKPOINTER
checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer) # send checkpointer while compiling the graph

In [13]:
config1 = {"configurable":{'thread_id': "1"}} # while executing persistence we send a thread id and against that thread id all the state will be stored 

workflow.invoke({"topic":"ai"}, config=config1)

{'topic': 'ai',
 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.',
 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, creating a humorous connection to therapy.'}

In [14]:
# THIS WILL GIVE THIS FINAL STATE
workflow.get_state(config1)

StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.', 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, creating a humorous connection to therapy.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b43c-6cc8-8002-dee0f0d2eaee'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-02T10:24:49.174413+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b20d-6bcb-8001-8c40dfcef62e'}}, tasks=(), interrupts=())

In [ ]:
# IF WE HAVE USED THE DATABASE MEMROY SAVER THEN AFTER MANY DAYS IF YOU RE RUN THE ABOVE CODE IT WILL GIVE THE ALLL THE SATE RESULT

In [15]:
# TO SEE INTERMEDIATE STATE VALUES
list(workflow.get_state_history(config1)) # we see 4 values for all the checkpointer

[StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.', 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, creating a humorous connection to therapy.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b43c-6cc8-8002-dee0f0d2eaee'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-02T10:24:49.174413+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b20d-6bcb-8001-8c40dfcef62e'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b20d-6bcb-8001-8c40dfcef62e'}}, metadata={'source': 'loop', 'step': 1, 'parents':

In [23]:
config2 = {"configurable":{'thread_id': "2"}} 

workflow.invoke({"topic":"ai"}, config=config2)

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" to work out, logically speaking.', 'explanation': 'The joke plays on the word "glitch," referencing both AI\'s technical issues and emotional problems to be worked out in therapy, creating a humorous pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f74f-c61a-6022-8002-658551b354e1'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-02T07:47:48.963077+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f74f-c3d6-6d39-8001-724f7736a456'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" to work out, logically speaking.'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f74f-c3d6-6d39-8001-724f7736a456'}}, metadata={'sour

### Benefits of Persistence in AI Systems

- **🔹 1. Short-Term Memory** : Essential for chatbots to maintain conversation context.


- **🔹 2. Fault Tolerance** : Allows resuming workflows from the last known state.      Stores intermediate data for recovery.


- **🔹 3. Time Travel (Debugging)** : Enables replaying past states.         Helps trace and fix bugs efficiently.


- **🔹 4. Human-in-the-Loop** : Maintains continuity during manual interventions.           Supports interactive AI workflows.

### 1. Fault Tolerance

In [5]:
import time
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [6]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [7]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:

# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}}) # we give none to tell we wamt to resume where it previously stopped
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...


EmptyInputError: Received no input for __start__

In [ ]:

list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

### Time Travel

In [16]:
# we will pass checkpoint id to get intermediate state
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06f8ae-b20d-6bcb-8001-8c40dfcef62e"}})

StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06f8ae-b20d-6bcb-8001-8c40dfcef62e'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-02T10:24:48.945420+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-ad1c-65be-8000-375907bea4ff'}}, tasks=(PregelTask(id='963458a5-c28b-25f5-12cc-ef051bdec551', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, creating a humorous connection to therapy.'}),), interrupts=())

In [ ]:
# and to start execution from the intermediate state and input state is given as none
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06f8ae-b20d-6bcb-8001-8c40dfcef62e"}})

{'topic': 'ai',
 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.',
 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, requiring therapy to resolve its thought process problems.'}

In [18]:
# we will see more states as we did time travel
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.', 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, requiring therapy to resolve its thought process problems.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8b1-7121-6922-8002-78b374991a84'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-02T10:26:02.668424+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8ae-b20d-6bcb-8001-8c40dfcef62e'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.', 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, creating a humorous connection to therapy.'}, next=(), config={'configurable': {'thread_id':

### Updating State

In [ ]:
# WE CAN CHANGE PARTICULATE STATE VALUE AT ANY CHECKPOINT (at last we changed the topic)

workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06f8bd-1120-6842-8000-76b96262123a'}}

In [20]:
# we will get additional state memory

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8bd-1120-6842-8000-76b96262123a'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-08-02T10:31:14.724222+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='a3450019-6460-ec6d-16ed-71c35e4e2dd1', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'ai', 'joke': 'Why did AI go to therapy? It had a little "glitch" in its thought process always.', 'explanation': 'The joke plays on "glitch" meaning both a technical AI error and a mental issue, requiring therapy to resolve its thought process problems.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': 

In [21]:
# Invoking for samosa
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06f8bd-1120-6842-8000-76b96262123a"}})

{'topic': 'samosa',
 'joke': 'Why was samosa in therapy? It was feeling a little "crunchy" under the pressure always.',
 'explanation': 'The joke plays on "crunchy" referring to both the samosa\'s texture and feeling overwhelmed under pressure, creating a humorous pun.'}

In [22]:

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why was samosa in therapy? It was feeling a little "crunchy" under the pressure always.', 'explanation': 'The joke plays on "crunchy" referring to both the samosa\'s texture and feeling overwhelmed under pressure, creating a humorous pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8c3-e050-6d55-8002-1f60498ebe79'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-02T10:34:17.510790+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8c3-dde6-644d-8001-377253eb0cf5'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why was samosa in therapy? It was feeling a little "crunchy" under the pressure always.'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06f8c3-dde6-644d-8001-377253eb0cf5'}}, metadata={'source': 